In [1]:
from rdflib import Graph
from rdflib.namespace import OWL, DC, DCTERMS, RDF, RDFS, SKOS, XSD
from rdflib import URIRef, BNode, Literal
import json
import os
import pandas as pd
import pylode

In [2]:
g = Graph()

##namespace
NS = "http://idss.org/terms/"
## create ontology
iseal = URIRef(NS)
g.add((iseal, RDF.type, OWL.Ontology)) 

In [4]:
df = pd.read_excel('/Users/marie-angeliquelaporte/Documents/jupyter/ISEAL/idss_schema_fields.xlsx')
df.dropna(how='all', axis=1)
df.fillna('', inplace=True)


for index, row in df.iterrows() :
    element_name = row['element name']
    element_description = row['element description']
    comment = row['element guidance']
    example = row['element link for more information']
    cardinality = row['element options']
    prop_type = row['element type']
    controlled_vocab = row['element controlled values or terms']
    module = row['idss module']
    module_cat = row['idss module category']
    
    ##module
    moduleUri = URIRef(NS+module)
    if not (None, SKOS.prefLabel, Literal(module)) in g:
        ##create module as skos concept
        g.add((moduleUri, RDF.type, OWL.Class)) ## SKOS.Concept
        g.add((moduleUri, SKOS.prefLabel, Literal(module)))
    ##element
    if '-' in element_name:
        concept = element_name.split(' - ')[0]
        element = element_name.split(' - ')[1]
        
        conceptUri = URIRef(NS+concept.replace(" ", "_"))
        if not (None, SKOS.prefLabel, Literal(concept)) in g:
            ##create concept as skos concept
            g.add((conceptUri, RDF.type, OWL.Class)) ## SKOS.Concept
            g.add((conceptUri, SKOS.prefLabel, Literal(concept)))
            g.add((conceptUri, RDFS.subClassOf, moduleUri))
        
        ## create properties
        elementURI = URIRef(NS+element.replace(" ", "_"))
        if prop_type == 'CONTROLLED VALUE': ## object property
            g.add((elementURI, SKOS.prefLabel, Literal(element)))
            g.add((elementURI, RDF.type, OWL.ObjectProperty))
            g.add((elementURI, RDFS.domain, conceptUri))
            
            ## create controlled vocab
            cvURI = URIRef(NS+"VOCAB_"+element.replace(" ", "_"))
            g.add((cvURI, RDF.type, OWL.Class)) ## SKOS.Concept ## SKOS.Collection??
            g.add((cvURI, SKOS.prefLabel, Literal("VOCAB "+element)))
            for term in controlled_vocab.split("||"):
                termURI = URIRef(NS+term.replace(" ", "_").replace("|", ""))
                g.add((termURI, RDF.type, OWL.Class)) ## SKOS.Concept
                g.add((termURI, SKOS.prefLabel, Literal(term)))
                g.add((termURI, RDFS.subClassOf, cvURI)) ## SKOS.member???
            g.add((elementURI, OWL.range, cvURI))
            
            ## cardinality
            if cardinality == 'MULTI SELECT FROM CONTROL LIST':
                br = BNode()
                g.add((br, RDF.type, OWL.Restriction))
                g.add((br, OWL.onProperty, elementURI))
                g.add((br, OWL.someValuesFrom, cvURI))
                g.add((conceptUri, RDFS.subClassOf, br))
            else:
                br = BNode()
                g.add((br, RDF.type, OWL.Restriction))
                g.add((br, OWL.onProperty, elementURI))
                g.add((br, OWL.maxQualifiedCardinality, Literal(1)))
                g.add((br, OWL.onClass, cvURI))
                g.add((conceptUri, RDFS.subClassOf, br))
            
        #elif prop_type == 'URL': ## object property
        #    g.add((elementURI, RDF.type, OWL.ObjectProperty))
        #    g.add((elementURI, OWL.domain, conceptUri))
        #    g.add((elementURI, OWL.range, URIRef("") ))
        #    g.add((elementURI, SKOS.prefLabel, Literal(element)))
        else: ## datatype properties
            g.add((elementURI, SKOS.prefLabel, Literal(element)))
            g.add((elementURI, RDF.type, OWL.DatatypeProperty))
            g.add((elementURI, RDFS.domain, conceptUri))
            range = None
            if prop_type == 'DATE':
                g.add((elementURI, RDFS.range, XSD.date ))
                range = XSD.date
            elif prop_type == 'NUMERIC VALUE':
                g.add((elementURI, RDFS.range, XSD.float))
                range = XSD.float
            else:
                g.add((elementURI, RDFS.range, XSD.string))
                range = XSD.string
            ##cardinality
            if cardinality == 'REPEAT VALUES':
                br = BNode()
                g.add((br, RDF.type, OWL.Restriction))
                g.add((br, OWL.onProperty, elementURI))
                g.add((br, OWL.someValuesFrom, range))
                g.add((conceptUri, RDFS.subClassOf, br))
            else:
                br = BNode()
                g.add((br, RDF.type, OWL.Restriction))
                g.add((br, OWL.onProperty, elementURI))
                g.add((br, OWL.maxQualifiedCardinality, Literal(1)))
                g.add((br, OWL.onDataRange, range))
                g.add((conceptUri, RDFS.subClassOf, br))
                
            
        if comment:
            g.add((elementURI, SKOS.scopeNote, Literal(comment)))
        if example:
            g.add((elementURI, RDFS.comment, Literal(example)))
        if element_description:
            g.add((elementURI, SKOS.definition, Literal(element_description)))
    else:
        print(element_name)
    
    
        
        

In [5]:
## save graph
g.serialize(destination='idds.ttl', format='turtle')

In [ ]:
html = pylode.MakeDocco(
    input_data_file="idds.ttl",
    outputformat="html",
    profile="ontdoc"
).document()

In [ ]:
f = open('documentation.html', 'w' )
f.write( html )
f.close()